In [1]:
from importlib import reload
import fitsio as fio
import numpy as np
import pandas as pd

import healpy as hp
import copy
import sys
import glob
import os
import matplotlib.pyplot as plt
%matplotlib inline

import sklearn.decomposition as decomp
import skysampler.emulator as emulator
import skysampler.indexer as indexer

import matplotlib as mpl
import subprocess as sp
import scipy.interpolate as interpolate
import pickle as pickle

import multiprocessing as mp

import skysampler.utils as utils


config read from file: /home/moon/vargatn/DES/Y3_WORK/skysampler-config/config/desy3-gamma-wide-config.yaml


# Some bad conversion from py2 pickle to py3...

In [6]:
fname = "/home/moon/vargatn/DES/PROJECTS/LSST/DATA/multi-indexer-epsilon_narrow-z_zoom_high-l_v004_clust_z0_l1_dict.p"

In [7]:
mdl_dict = pickle.load(open(fname, "rb"), encoding="latin1")

In [8]:
mdl = indexer.IndexedDataContainer(**mdl_dict)

In [10]:
rname = "/home/moon/vargatn/DES/PROJECTS/LSST/DATA/multi-indexer-epsilon_narrow-z_zoom_high-l_v004_rands_z0_l1_dict.p"

In [11]:
mdl_rands_dict = pickle.load(open(rname, "rb"), encoding="latin1")
mdl_rands = indexer.IndexedDataContainer(**mdl_rands_dict)

In [13]:
table = pd.read_hdf("/home/moon/vargatn/DES/PROJECTS/LSST/DATA/cosmoDC2_v1.1.4_refpixel_morepars-v2-8786.h5", key="data")

In [17]:
np.sort(table.keys())

array(['Mag_true_g_lsst_z0', 'Mag_true_i_lsst_z0', 'Mag_true_r_lsst_z0',
       'Mag_true_u_lsst_z0', 'Mag_true_y_lsst_z0', 'Mag_true_z_lsst_z0',
       'dec', 'ellipticity_1_true', 'ellipticity_2_true', 'galaxy_id',
       'halo_id', 'halo_mass', 'is_central', 'mag_g', 'mag_i', 'mag_r',
       'mag_u', 'mag_y', 'mag_z', 'ra', 'redshift_true', 'shear_1',
       'shear_2', 'size_bulge_true', 'size_disk_true',
       'size_minor_bulge_true', 'size_minor_disk_true', 'size_minor_true',
       'size_true', 'stellar_mass'], dtype=object)

In [43]:
deep_smc_settings = {
    "columns": [
        ("GABS", ("ellipticity_1_true", "ellipticity_2_true", "SQSUM")),
        ("SIZE", "size_true"),
        ("SIZE_MINOR", "size_minor_true"),
        ("SIZE_BULGE", "size_bulge_true"),
        ("SIZE_MINOR_BULGE", "size_minor_bulge_true"),
        ("SIZE_DISK", "size_disk_true"),
        ("SIZE_MINOR_DISK", "size_minor_disk_true"),
        ("MAG_I", "mag_i"),
        ("COLOR_G_R", ("mag_g", "mag_r", "-")),
        ("COLOR_R_I", ("mag_r", "mag_i", "-")),
        ("COLOR_I_Z", ("mag_i", "mag_z", "-")),
        ("STELLAR_MASS", "stellar_mass"),
    ],
    "logs": [False, True, True, True, True, True, True, False, False, False, False, True],
    "limits": [(0., 1.), (-1, 5), (-1, 5), (-1, 5), (-1, 5), (-1, 5), (-1, 5), (17, 25), (-1, 3), (-1, 3), (-1, 3), (10**3, 10**13)],
}

In [44]:
len(deep_smc_settings["logs"])

12

In [45]:
len(deep_smc_settings["limits"])


12

In [46]:
len(deep_smc_settings["columns"])

12

In [9]:
mdl.samples

[    COADD_OBJECT_ID          RA        DEC  BPZ_ZMC_SOF  DNF_ZMC_SOF  \
 0         312169742   39.492643 -18.417508     0.353398     0.307438   
 1         472784116   68.388805 -37.769073     0.412624     0.335321   
 2         199813915    5.914300 -35.807501     0.389930     0.327795   
 3         125679001  350.673699 -40.458893     0.375996     0.292500   
 4         262078735   28.980643  -8.987514     0.338538     0.302018   
 ..              ...         ...        ...          ...          ...   
 93        310782944   39.680567 -46.674285     0.419273     0.293115   
 94        310405853   39.147823 -59.215161     0.402497     0.366364   
 95        168658650  306.371322 -55.154342     0.392521     0.326308   
 96        143256954    3.330515 -41.243120     0.366337     0.344692   
 97        242689529  328.300073 -41.826003     0.441200     0.335414   
 
     FLAGS_GOLD  MOF_CM_FLAGS  MOF_FLAGS  MOF_CM_G_1  MOF_CM_G_2  ...  \
 0            0             0          0   -0.053